In [242]:
import textworld
import glob
import jericho
from jericho import FrotzEnv
from jericho.util import get_subtree
from collections import defaultdict
import pandas as pd
import nltk

# GAME_FOLDER = "/Users/SXH1M01/Data/the-large-game-collection"
GAME_FOLDER = "/Users/SXH1M01/GitHub/z-machine-games/jericho-game-suite"
files = glob.glob(f"{GAME_FOLDER}/*")
len(files)

57

In [73]:
env = FrotzEnv(f"{GAME_FOLDER}/zork1.z5")
env.reset()

('Copyright (c) 1981, 1982, 1983 Infocom, Inc. All rights reserved.\nZORK is a registered trademark of Infocom, Inc.\nRevision 88 / Serial number 840726\n\nWest of House\nYou are standing in an open field west of a white house, with a boarded front door.\nThere is a small mailbox here.\n\n',
 {'moves': 0, 'score': 0})

In [74]:
objs = list(env.get_world_objects())
objs[0:10]

[Obj0:  Parent0 Sibling0 Child0 Attributes [] Properties [],
 Obj1: pair hands Parent247 Sibling2 Child0 Attributes [14, 28] Properties [18, 16],
 Obj2: zorkmid Parent247 Sibling3 Child0 Attributes [] Properties [18, 17],
 Obj3: way Parent247 Sibling5 Child0 Attributes [14] Properties [18, 17, 16],
 Obj4: cretin Parent180 Sibling181 Child0 Attributes [7, 9, 14, 30] Properties [18, 17, 7],
 Obj5: you Parent247 Sibling6 Child0 Attributes [30] Properties [18, 17],
 Obj6: blast air Parent247 Sibling7 Child0 Attributes [14] Properties [18],
 Obj7: lurking grue Parent247 Sibling8 Child0 Attributes [] Properties [18, 17, 16],
 Obj8: ground Parent247 Sibling9 Child0 Attributes [] Properties [18, 17],
 Obj9: sailor Parent247 Sibling10 Child0 Attributes [14] Properties [18, 17]]

In [244]:
def obj2dict(obj):
    o = {
        "name": obj.name.strip(),
        "id": obj.num,
        "parent": obj.parent,
        "sibling": obj.sibling,
        "attr": list(obj.attr),
        "prop": list(obj.properties)
    }    
    return o

class GameGraph(object):
    def __init__(self, env, id2obj, game_log, walkthrough, locations, location_transitions, location_items, gettable_items):
        env.reset()
        self.env = env
        self.id2obj = id2obj
        self.game_log = game_log
        self.walkthrough = walkthrough
        self.locations = locations
        self.location_transitions = location_transitions
        self.location_items = location_items
        self.gettable_items = gettable_items

In [246]:
game2gg = dict()

for fname in files:
    
    jenv = FrotzEnv(fname)

    walkthrough = None
    initial_observation, info = jenv.reset()
    try:
        walkthrough = jenv.get_walkthrough()
    except:
        print(f"Error - skipping: {fname}")
        continue

    objs = list(jenv.get_world_objects())
    id2obj = dict()
    for obj in jenv.get_world_objects():
        id2obj[obj.num] = obj2dict(obj)

    locns = set()
    inv_objs = set()
    log = [{
        "act": "start",
        "obs": initial_observation.strip(),
        "inv": [],
        "valid_actions": [],
        "reward": 0
    }]
    current_locn = -1
    location_transitions = defaultdict(lambda : defaultdict(int))
    location_items = defaultdict(set)
    prev_inv = set()
    picked_up_items = set()
    for act in walkthrough:   
        observation, reward, done, info = jenv.step(act)
        locn = jenv.get_player_location()
        if locn:            
            if locn.num != current_locn:
                locns.add(locn.num)
                location_transitions[current_locn][locn.num] += 1
                current_locn = locn.num
                
                objs = get_subtree(current_locn,g.env.get_world_objects())
                items = [o.num for o in objs]
                location_items[current_locn].update(items)

        else:
            current_locn = -1

    # Takes a long time
    #     valid_acts = jenv.get_valid_actions(use_object_tree=True)
        current_inv = {o.num for o in jenv.get_inventory()}
        inv_added = current_inv - prev_inv
        inv_removed = prev_inv - current_inv
        prev_inv = current_inv
        for num in inv_added:
            if num not in picked_up_items:
                picked_up_items.add(num)


        log.append({
           "act": act,
            "locn": locn.num if locn else -1,
            "obs": observation.strip(),
            "inv": current_inv,
            "valid_actions": [],
            "reward": reward,
            "info": info
        })
    
    game_graph = GameGraph(env=jenv, id2obj=id2obj, game_log=log, walkthrough=walkthrough, 
                           locations=locns, location_transitions=location_transitions, 
                           location_items=location_items, gettable_items=picked_up_items)


    game_name = fname.split("/")[-1].split(".")[0]
    game2gg[game_name] = game_graph
    if len(game2gg) % 5 == 0:
        print(len(game2gg))

5
10
15
20
25
30
35
40
45
50
55


In [247]:
all_transitions = defaultdict(lambda: defaultdict(int))
for game, g in game2gg.items():
    for a,tally in g.location_transitions.items():
        if a == -1:
            continue
        for b,freq in tally.items():            
            aname = g.id2obj[a]["name"].lower().strip()
            bname = g.id2obj[b]["name"].lower().strip()
            if aname == bname:
                continue
            all_transitions[aname][bname] += freq
            
for a,tally in sorted(all_transitions.items())[0:10]:
    for b,cnt in sorted(tally.items(), key = lambda tpl: (-tpl[1],tpl[0]))[0:5]:
        if cnt == 1:
            continue
        print(cnt, " ", a.ljust(30),"->",b)

4                                  -> shanty
3                                  -> int'l
3                                  -> strange passage
3                                  -> vacant lot
2                                  -> ocean
2   's lair                        -> old winery
2   's lair                        -> pott room


In [66]:
for a, tally in location_transitions.items():
    if a == -1:
        continue
    for b,cnt in tally.items():
        print(f"{id2obj[a]['name']}->{id2obj[b]['name']} : {cnt}")

North House->Forest Path : 2
North House->Behind House : 4
North House->West House : 1
Forest Path->Up a Tree : 1
Forest Path->North House : 4
Forest Path->Forest : 1
Up a Tree->Forest Path : 1
Behind House->Kitchen : 5
Behind House->North House : 2
Kitchen->Living : 6
Kitchen->Attic : 1
Kitchen->Behind House : 2
Living->Kitchen : 3
Living->Cellar : 8
Living->Strange Passage : 2
Attic->Kitchen : 1
Cellar->Troll : 8
Cellar->East Chasm : 1
Cellar->Living : 4
Troll->East-West Passage : 7
Troll->Cellar : 3
Troll->Maze : 1
East-West Passage->Round : 7
East-West Passage->Troll : 3
Round->Engravings Cave : 2
Round->Loud : 2
Round->Narrow Passage : 3
Round->East-West Passage : 3
Round->North-South Passage : 1
Engravings Cave->Dome : 2
Dome->Torch : 2
Torch->Temple : 2
Temple->Altar : 3
Temple->Egypti : 2
Altar->Temple : 1
Altar->Forest : 2
Egypti->Temple : 2
Forest->Forest Path : 2
East Chasm->Gallery : 1
East Chasm->Cellar : 1
Gallery->East Chasm : 1
Loud->Deep Canyon : 1
Loud->Round : 1
Loud

In [67]:
for locn, items in location_items.items():
    print(f"{id2obj[locn]['name'].upper()}")
    for item in items:
        print(f"{id2obj[item]['name']}")
    print()

UP A TREE
jewel-encrusted egg

KITCHEN
clove garlic

LIVING
brass lantern
ancient map
sword
golden clockwork canary

ATTIC
rope
nasty knife

TORCH
torch

ALTAR
pair candles
black book

TEMPLE
brass bell

EGYPTI
gold coffin
sceptre

GALLERY
painting

LOUD
platinum bar

DAM LOBBY
matchbook

LDEAD
crystal skull

MAZE
rusty knife
leather bag coins

DAM BASE
pile plastic

MAINTENANCE
wrench
screwdriver

ATLANT
crystal trident

RESERVOIR NORTH
hand-held air pump

WHITE CLIFFS BEACH
tlabel

MAGIC BOAT
red buoy

SANDY BEACH
shovel
emerald

SANDY CAVE
beautiful jeweled scarab

END RAINBOW
pot gold

TREASURE
trunk jewels
chalice

FOREST
beautiful brass bauble

DEAD END
pile coal

MACHINE
huge diamond

GAS
sapphire-encrusted bracelet

BAT
jade figurine



In [68]:
for l in locns:
    print(l, id2obj[l]["name"])

15 Slide
16 Coal Mine
17 Coal Mine
18 Coal Mine
19 Coal Mine
20 Ladder Bottom
21 Ladder Top
22 Smelly
23 Squeaky
24 Mine Entrance
25 CanyView
26 Rocky Ledge
27 CanyBottom
28 On Rainbow
29 AragaFalls
30 Shore
33 White Cliffs Beach
37 Chasm
38 North-South Passage
39 Damp Cave
40 Deep Canyon
41 East-West Passage
44 Narrow Passage
45 Cold Passage
46 Cave
47 Cave
49 Stream View
50 Reservoir South
51 Strange Passage
52 Maze
63 Maze
64 Maze
67 Maze
68 Maze
70 Maze
71 East Chasm
72 Cellar
74 Clearing
75 Forest Path
77 Forest
78 Forest
79 Behind House
81 North House
88 Up a Tree
96 Engravings Cave
100 Reservoir
102 Troll
105 Torch
107 Round
118 Dead End
120 Sandy Beach
124 Gas
126 Sandy Cave
133 Dome
136 End Rainbow
138 Loud
140 Dam Base
148 Gallery
150 Mirror
152 Mirror
154 Dam Lobby
156 magic boat
157 Machine
167 Maze
172 Reservoir North
175 Egypti
178 Stone Barrow
180 West House
185 
187 Atlant
190 Treasure
193 Living
199 Maintenance
201 Attic
203 Kitchen
206 Timber
212 Altar
215 Dam
220 Tem

In [228]:
def print_log(g):
    for entry in g.game_log:
        print("*" * 80)
        print("Act:",entry["act"])
        print(entry["obs"].strip())

In [229]:
g = game2gg['acorncourt']
print_log(g)

********************************************************************************
Act: start
GREAT.  THEY'VE DONE IT TO ME AGAIN.  You think to yourself. THEY'VE STUCK ME IN ANOTHER ONE OF THEIR SILLY SCENARIOS.  You glance about with a look of irritation on your face.  WELL, I'LL SHOW THEM.  I'LL MAKE SHORT WORK OF THEIR STUPID LITTLE PUZZLE...

THE ACORN COURT
An Interactive Text Adventure
Copyright (c) 1997 by Todd S. Murchison.
Release 3 / Serial number 970904 / Inform v6.13 Library 6/5
Standard interpreter 1.0

Court Yard
A good sized courtyard with an air of late British colonialism about it.  To the west, in the direction the sun is setting, is a high, grey stone wall with an ornate iron gate set into the rock.  The walls of a large stone mansion rise several stories into the chilly evening air to the east, north, and south.  In the east wall, two or three stories up, is a large window.  On top of the west wall, above and to the right of the gate, is a large squirrels nest made o

In [185]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [194]:
dct = {str(w).lower().strip() for w in g.env.get_dictionary()}
text = g.game_log[0]['obs']
doc = nlp(text)
doc_tokens = set()
for t in doc:
    doc_tokens.add(str(t).lower())

In [217]:
objs = set()
for t in doc:
    tok = str(t).lower().strip()
    if tok not in dct:
        continue
    if t.pos_ == "NOUN":
        objs.add(tok)
# objs

In [210]:
e = g.env
e.reset()

("\nGREAT.  THEY'VE DONE IT TO ME AGAIN.  You think to yourself. THEY'VE STUCK ME IN ANOTHER ONE OF THEIR SILLY SCENARIOS.  You glance about with a look of irritation on your face.  WELL, I'LL SHOW THEM.  I'LL MAKE SHORT WORK OF THEIR STUPID LITTLE PUZZLE...\n\nTHE ACORN COURT\nAn Interactive Text Adventure\nCopyright (c) 1997 by Todd S. Murchison.\nRelease 3 / Serial number 970904 / Inform v6.13 Library 6/5\nStandard interpreter 1.0\n\nCourt Yard\nA good sized courtyard with an air of late British colonialism about it.  To the west, in the direction the sun is setting, is a high, grey stone wall with an ornate iron gate set into the rock.  The walls of a large stone mansion rise several stories into the chilly evening air to the east, north, and south.  In the east wall, two or three stories up, is a large window.  On top of the west wall, above and to the right of the gate, is a large squirrels nest made of sticks, twigs, and leaves.\n\nA carpeting of old brown leaves from past winte

In [172]:
sorted(game2gg.keys())

['905',
 'acorncourt',
 'advent',
 'adventureland',
 'afflicted',
 'anchor',
 'awaken',
 'balances',
 'ballyhoo',
 'curses',
 'cutthroat',
 'deephome',
 'detective',
 'dragon',
 'enchanter',
 'enter',
 'gold',
 'hhgg',
 'hollywood',
 'huntdark',
 'infidel',
 'inhumane',
 'jewel',
 'karn',
 'lgop',
 'library',
 'loose',
 'lostpig',
 'ludicorp',
 'lurking',
 'moonlit',
 'murdac',
 'night',
 'omniquest',
 'partyfoul',
 'pentari',
 'planetfall',
 'plundered',
 'reverb',
 'seastalker',
 'sherlock',
 'snacktime',
 'sorcerer',
 'spellbrkr',
 'spirit',
 'temple',
 'theatre',
 'trinity',
 'tryst205',
 'weapon',
 'wishbringer',
 'yomomma',
 'zenon',
 'zork1',
 'zork2',
 'zork3',
 'ztuu']

In [232]:
for l in g.locations:
    print(g.id2obj[l]["name"])

In the garden
In the cellar
In a dark passage
In a dungeon
In the bedroom
At the top of the beanstalk
At a fork in the path
In the potting shed
In the kitchen
In Papa Bear's workshop
In the hall
In the pantry
On a path in the forest
In a small meadow
In a clearing in the forest
In the enchanted forest
In the sitting room
large chair
Inside the medium-sized chair
On the porch
In a cave
On the roof


In [ ]:
'jump to roof'

In [230]:
g = game2gg["gold"]
print_log(g)

********************************************************************************
Act: start
-------------------------
By J. J. Guest.
-------------------------
ADRIFT release 1.2 16th September 2002
Visit J. J. Guest's website at: www.groundchuck.co.uk
Inform port by ralphmerridew (ralphmerridew@gmail.com)
Inform version 1.0, April 2006
Type ZOOM IN, ZOOM OUT, or ZOOM AWAY to toggle automapping.  (Do not use with Gargoyle due to bugs.)

Man, what a party! That was a night to remember - actually it'll be a miracle if I remember it at all tomorrow!  Oh, by the way, I'm Goldilocks. I'm a nineteen year old blonde bombshell and I'm on my way back from yet another crazy art school party. It's about five AM and the sun's already up, as are the birds, who are doing their level best to aggravate my hangover with their bloody singing! I could really use something to eat and somewhere to sleep off this hangover, but something tells me I shouldn't have taken this shortcut through the forest. I hav

In [233]:
e.get_state()

(array([  5, 128,   0, ...,   0,   0,   0], dtype=uint8),
 array([ 0,  0,  0, ...,  0, 22,  0], dtype=uint8),
 21876,
 943,
 950,
 7,
 (1169043488, 0, 0),
 b"The machine is old and incrusted with dirt, but it looks as if it might still work.\nThe device spitters to life in a noisy cacophony of pseudo-humorous mechanical sounds.  The launching spring begins to coil back into place.\nA tennis ball launches out of the machine driven by the machine's old fashioned spring mechanism.  The ball lands uselessly on the ground.")

[39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53]

In [236]:
g.id2obj[39]

{'name': 'In the kitchen',
 'id': 39,
 'parent': 0,
 'sibling': 0,
 'attr': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'prop': [38, 35, 9, 7, 4, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [235]:
g.locations

{39,
 54,
 60,
 74,
 77,
 92,
 93,
 106,
 108,
 114,
 127,
 128,
 134,
 146,
 147,
 156,
 167,
 168,
 169,
 179,
 189,
 190}